In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoModelForCausalLM, AutoModel, AutoTokenizer
import nltk
import json
import pandas as pd
from datasets import *
import numpy as np
import torch
from tqdm import tqdm

nltk.download('punkt')
SEED = 42

/home/valter/conda/envs/nlp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to /home/valter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
split = "test"
data_path_pairs = '../../data/processed/prompt_reply_pairs.csv'
num_return_sequences = 5

In [3]:
# model_checkpoint = "cjvt/t5-sl-small"
# model_checkpoint = "ls1906/t5-sl-small-finetuned-assistant"

# model_checkpoint = "cjvt/gpt-sl-base"
# model_checkpoint = "vh-student/gpt-sl-oasst1-pairs"
# model_checkpoint = ...

model_name = model_checkpoint.split("/")[-1]

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
if "t5" in model_name:
    model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint).to("cuda")
elif "gpt" in model_name:
    model = AutoModelForCausalLM.from_pretrained(model_checkpoint).to("cuda")
else:
    raise ValueError("Model not supported")

model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(60032, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): FastGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=60032, bias=False)
)

In [4]:
data = pd.read_csv(data_path_pairs, sep=";")
data

,prompt,reply,rank,split
0,"V mojo hišo bodo prispeli 3 prijatelji, rad bi...",Kaj pa igranje lažnivca? Za to igro so potrebn...,0,train
1,"V mojo hišo bodo prispeli 3 prijatelji, rad bi...","Uno: Je klasična igra za otroke in odrasle, pr...",1,train
2,"V mojo hišo bodo prispeli 3 prijatelji, rad bi...",Igrate lahko:\n - 8 noro\n - celina\n - Lažnivec,2,train
3,Napišite hvalnico šolskemu taboru v slogu Mela...,Spomnim se vas prijatelji\nIn spet sva se sreč...,0,train
4,Ali kaj bolj veselega?,♪ Spakirajte in se pridružite zabavi!♪\n♪ Šols...,0,train
...,...,...,...,...
47509,"Hvala, bom videl, če lahko naredim te sprememb...",Elektronski seti bobnov so na splošno tišji od...,0,test
47510,"Hvala, bom videl, če lahko naredim te sprememb...","To je odvisno od seta bobnov, ki ga trenutno u...",1,test
47511,"Imam elektronski set bobnov, a vsakič, ko igra...",Pod komplet lahko poskusite postaviti izolacij...,2,test
47512,Kadarkoli poskušam postaviti izolacijski mater...,"Da, obstaja več načinov, kako to popraviti.\n\...",0,test


In [5]:
data[data["split"] == split][:10]

,prompt,reply,rank,split
42639,Vzemimo naključno matriko $n\krat n$ z realnim...,"Ocenjevanje natančne verjetnosti, da imajo vse...",0,test
42640,Hvala vam! Jasno mi je,Vedno z veseljem pomagamo! Vas zanima še kaj?,0,test
42641,Hvala vam! Jasno mi je,prosim!\nV veselje mi je bilo pomagati vam. Ve...,1,test
42642,Hvala vam! Jasno mi je,Z veseljem pomagam! Ali potrebujete še kaj?,2,test
42643,Kakšen je geometrijski pomen izpeljanke?,"Geometrični pomen odvoda je, da pokaže, kako h...",0,test
42644,Kakšen je geometrijski pomen izpeljanke?,Vrednost odvoda (realne) funkcije (realnega ar...,1,test
42645,Vzemimo naključno matriko $n\krat n$ z realnim...,"Verjetnost, da imajo lastne vrednosti naključn...",1,test
42646,"Recimo, da je samovozeči avto v situaciji, ko ...",Avtonomni avtomobili morajo biti programirani ...,0,test
42647,"Recimo, da je samovozeči avto v situaciji, ko ...","Avto mora biti programiran tako, da zagotavlja...",1,test
42648,"Recimo, da je samovozeči avto v situaciji, ko ...","To vrsto etičnih dilem, kot je tista, ki jo pr...",2,test


In [6]:
eval_data = Dataset.from_pandas(data[data['split'] == split][["prompt", "reply"]])
eval_data

Dataset({
    features: ['prompt', 'reply', '__index_level_0__'],
    num_rows: 4875
})

In [7]:
eval_data[:10]["prompt"]

['Vzemimo naključno matriko $n\\krat n$ z realnimi elementi, od katerih je vsak element izbran neodvisno od drugih iz standardne normalne porazdelitve. Kako oceniti verjetnost, da imajo vse lastne vrednosti te matrike pozitiven realni del? Kakšno je asimptotično obnašanje te verjetnosti, ko $n$ teži k neskončnosti?',
 'Hvala vam! Jasno mi je',
 'Hvala vam! Jasno mi je',
 'Hvala vam! Jasno mi je',
 'Kakšen je geometrijski pomen izpeljanke?',
 'Kakšen je geometrijski pomen izpeljanke?',
 'Vzemimo naključno matriko $n\\krat n$ z realnimi elementi, od katerih je vsak element izbran neodvisno od drugih iz standardne normalne porazdelitve. Kako oceniti verjetnost, da imajo vse lastne vrednosti te matrike pozitiven realni del? Kakšno je asimptotično obnašanje te verjetnosti, ko $n$ teži k neskončnosti?',
 'Recimo, da je samovozeči avto v situaciji, ko sta dve možnosti: povoziti starca in rešiti otroka ali povoziti otroka, da bi rešil starca, kako naj bo avto programiran?',
 'Recimo, da je sam

In [8]:
def convert_to_features(examples):
    prefix_in = "Uporabnik: "
    # prefix_in = ""
    examples["prompt"] = [prefix_in + prompt for prompt in examples["prompt"]]
    # prefix_out = "Asistent: "
    prefix_out = ""
    examples["reply"] = [prefix_out + reply for reply in examples["reply"]]
    
    model_inputs = tokenizer(examples['prompt'], pad_to_max_length=True, max_length=512, truncation=True, return_tensors='pt')

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['reply'], pad_to_max_length=True, max_length=128, truncation=True, return_tensors='pt')

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [9]:
eval_data = eval_data.map(convert_to_features, batched=True, load_from_cache_file=False)
eval_data.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
eval_data

Map:   0%|          | 0/4875 [00:00<?, ? examples/s]/home/valter/conda/envs/nlp/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/home/valter/conda/envs/nlp/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3606: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  w

Dataset({
    features: ['prompt', 'reply', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 4875
})

### Generate answers

The generation ran almost 30minutes

In [10]:
# split so it fits on a 12gb gpu - couldn't find a better way (model.generate() options that would do this automatically...)
# depends on the model
step = 50 # makeshift batch size - idk how to do this better
indices = np.concatenate([np.arange(0, eval_data.num_rows, step), [eval_data.num_rows]])

outputs = []
model = model.to("cuda")
with torch.no_grad():
    with torch.cuda.amp.autocast(True):
        for i in tqdm(range(len(indices[:-1]))):
            generated = model.generate(eval_data[indices[i]:indices[i+1]]['input_ids'].to("cuda"),
                                    attention_mask=eval_data[indices[i]:indices[i+1]]['attention_mask'].to("cuda"),
                                    max_new_tokens=128, no_repeat_ngram_size=2, num_beams=5, num_return_sequences=num_return_sequences, early_stopping=True)
            outputs.append(generated)

100%|██████████| 98/98 [1:25:34<00:00, 52.39s/it]


In [11]:
# results = tokenizer.batch_decode(outputs, skip_special_tokens=True)
results, prediction_lens = [], []
for output in outputs:
    results.extend(tokenizer.batch_decode(output, skip_special_tokens=True))
    prediction_lens.extend([(np.count_nonzero(o.cpu() != tokenizer.pad_token_id)) for o in output])

In [12]:
result_data = pd.read_csv(data_path_pairs, sep=";")

# repeat rows for the number of generated replies
temp = pd.DataFrame(np.repeat(result_data.values, num_return_sequences, axis=0))
temp.columns = result_data.columns
result_data = temp

result_data = result_data[result_data['split'] == split].reset_index(drop=True)
result_data["generated"] = results
result_data["token_prediction_len"] = prediction_lens
result_data.to_csv(f"../../data/results/prompt_reply_pairs_{num_return_sequences}_generated_{split}_{model_name}.csv", sep=";", index=False)
display(result_data)

result_data_1_gen = result_data[::num_return_sequences]
result_data_1_gen.to_csv(f"../../data/results/prompt_reply_pairs_1_generated_{split}_{model_name}.csv", sep=";", index=False)
display(result_data_1_gen)

,prompt,reply,rank,split,generated,token_prediction_len
0,Vzemimo naključno matriko $n\krat n$ z realnim...,"Ocenjevanje natančne verjetnosti, da imajo vse...",0,test,Uporabnik: Vzemimo naključno matriko $n\krat ...,208
1,Vzemimo naključno matriko $n\krat n$ z realnim...,"Ocenjevanje natančne verjetnosti, da imajo vse...",0,test,Uporabnik: Vzemimo naključno matriko $n\krat ...,208
2,Vzemimo naključno matriko $n\krat n$ z realnim...,"Ocenjevanje natančne verjetnosti, da imajo vse...",0,test,Uporabnik: Vzemimo naključno matriko $n\krat ...,208
3,Vzemimo naključno matriko $n\krat n$ z realnim...,"Ocenjevanje natančne verjetnosti, da imajo vse...",0,test,Uporabnik: Vzemimo naključno matriko $n\krat ...,208
4,Vzemimo naključno matriko $n\krat n$ z realnim...,"Ocenjevanje natančne verjetnosti, da imajo vse...",0,test,Uporabnik: Vzemimo naključno matriko $n\krat ...,208
...,...,...,...,...,...,...
24370,Kadarkoli poskušam postaviti izolacijski mater...,"Težava, da udarec pobegne, je precej pogosta p...",1,test,Uporabnik: Kadarkoli poskušam postaviti izola...,218
24371,Kadarkoli poskušam postaviti izolacijski mater...,"Težava, da udarec pobegne, je precej pogosta p...",1,test,Uporabnik: Kadarkoli poskušam postaviti izola...,218
24372,Kadarkoli poskušam postaviti izolacijski mater...,"Težava, da udarec pobegne, je precej pogosta p...",1,test,Uporabnik: Kadarkoli poskušam postaviti izola...,218
24373,Kadarkoli poskušam postaviti izolacijski mater...,"Težava, da udarec pobegne, je precej pogosta p...",1,test,Uporabnik: Kadarkoli poskušam postaviti izola...,218


,prompt,reply,rank,split,generated,token_prediction_len
0,Vzemimo naključno matriko $n\krat n$ z realnim...,"Ocenjevanje natančne verjetnosti, da imajo vse...",0,test,Uporabnik: Vzemimo naključno matriko $n\krat ...,208
5,Hvala vam! Jasno mi je,Vedno z veseljem pomagamo! Vas zanima še kaj?,0,test,"Uporabnik: Hvala vam! Jasno mi je, da v tem t...",139
10,Hvala vam! Jasno mi je,prosim!\nV veselje mi je bilo pomagati vam. Ve...,1,test,"Uporabnik: Hvala vam! Jasno mi je, da v tem t...",139
15,Hvala vam! Jasno mi je,Z veseljem pomagam! Ali potrebujete še kaj?,2,test,"Uporabnik: Hvala vam! Jasno mi je, da v tem t...",139
20,Kakšen je geometrijski pomen izpeljanke?,"Geometrični pomen odvoda je, da pokaže, kako h...",0,test,Uporabnik: Kakšen je geometrijski pomen izpel...,139
...,...,...,...,...,...,...
24350,"Hvala, bom videl, če lahko naredim te sprememb...",Elektronski seti bobnov so na splošno tišji od...,0,test,"Uporabnik: Hvala, bom videl, če lahko naredim...",201
24355,"Hvala, bom videl, če lahko naredim te sprememb...","To je odvisno od seta bobnov, ki ga trenutno u...",1,test,"Uporabnik: Hvala, bom videl, če lahko naredim...",201
24360,"Imam elektronski set bobnov, a vsakič, ko igra...",Pod komplet lahko poskusite postaviti izolacij...,2,test,"Uporabnik: Imam elektronski set bobnov, a vsa...",173
24365,Kadarkoli poskušam postaviti izolacijski mater...,"Da, obstaja več načinov, kako to popraviti.\n\...",0,test,Uporabnik: Kadarkoli poskušam postaviti izola...,218


In [13]:
# remove the duplicate prompts (we generate the same reply but we have them because during training they have different replies)
# also only keep the prompt and generated columns
result_data = result_data.drop_duplicates(subset=["prompt"])[["prompt", "generated"]]
result_data.to_csv(f"../../data/results/prompt_reply_pairs_{num_return_sequences}_generated_{split}_{model_name}_no_duplicates.csv", sep=";", index=False)
display(result_data)

result_data_1_gen = result_data_1_gen.drop_duplicates(subset=["prompt"])[["prompt", "generated"]]
result_data_1_gen.to_csv(f"../../data/results/prompt_reply_pairs_1_generated_{split}_{model_name}_no_duplicates.csv", sep=";", index=False)
display(result_data_1_gen)

,prompt,generated
0,Vzemimo naključno matriko $n\krat n$ z realnim...,Uporabnik: Vzemimo naključno matriko $n\krat ...
5,Hvala vam! Jasno mi je,"Uporabnik: Hvala vam! Jasno mi je, da v tem t..."
20,Kakšen je geometrijski pomen izpeljanke?,Uporabnik: Kakšen je geometrijski pomen izpel...
35,"Recimo, da je samovozeči avto v situaciji, ko ...","Uporabnik: Recimo, da je samovozeči avto v si..."
50,"Želim pripraviti pico s feferoni, mi lahko nav...","Uporabnik: Želim pripraviti pico s feferoni, ..."
...,...,...
24315,Mi lahko poveste več informacij o Franciscu Mo...,Uporabnik: Mi lahko poveste več informacij o ...
24330,"Mi lahko poveste, kateri so glavni rezultati a...","Uporabnik: Mi lahko poveste, kateri so glavni..."
24340,"Imam elektronski set bobnov, a vsakič, ko igra...","Uporabnik: Imam elektronski set bobnov, a vsa..."
24350,"Hvala, bom videl, če lahko naredim te sprememb...","Uporabnik: Hvala, bom videl, če lahko naredim..."


,prompt,generated
0,Vzemimo naključno matriko $n\krat n$ z realnim...,Uporabnik: Vzemimo naključno matriko $n\krat ...
5,Hvala vam! Jasno mi je,"Uporabnik: Hvala vam! Jasno mi je, da v tem t..."
20,Kakšen je geometrijski pomen izpeljanke?,Uporabnik: Kakšen je geometrijski pomen izpel...
35,"Recimo, da je samovozeči avto v situaciji, ko ...","Uporabnik: Recimo, da je samovozeči avto v si..."
50,"Želim pripraviti pico s feferoni, mi lahko nav...","Uporabnik: Želim pripraviti pico s feferoni, ..."
...,...,...
24315,Mi lahko poveste več informacij o Franciscu Mo...,Uporabnik: Mi lahko poveste več informacij o ...
24330,"Mi lahko poveste, kateri so glavni rezultati a...","Uporabnik: Mi lahko poveste, kateri so glavni..."
24340,"Imam elektronski set bobnov, a vsakič, ko igra...","Uporabnik: Imam elektronski set bobnov, a vsa..."
24350,"Hvala, bom videl, če lahko naredim te sprememb...","Uporabnik: Hvala, bom videl, če lahko naredim..."
